In [9]:
import ipywidgets as widgets
from IPython.display import display
from groq import Groq
import torch
from transformers import pipeline
import torchaudio

In [10]:
import pyttsx3

def speak_text(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 170)  # Moderate speed
    engine.setProperty('volume', 1.0)  # Max volume

    # Choose a clearer voice (index may vary by OS)
    voices = engine.getProperty('voices')
    if len(voices) > 1:
        engine.setProperty('voice', voices[1].id)  # Female (or clearer) voice
    else:
        engine.setProperty('voice', voices[0].id)

    engine.say(text)
    engine.runAndWait()


In [11]:
def build_prompt(topic: str, duration: int, emotion: str = "formal", audience: str = "general") -> str:
    """
    Build a detailed prompt for Groq-based speech generation.
    """
    style_templates = {
        "formal": "Write a formal {duration}-minute speech about '{topic}' suitable for a professional audience.",
        "casual": "Write a casual, friendly {duration}-minute speech about '{topic}'.",
        "motivational": "Write an inspiring {duration}-minute motivational speech about '{topic}' that energizes the audience.",
        "persuasive": "Write a compelling {duration}-minute persuasive speech about '{topic}' to change minds.",
        "instructional": "Write a step-by-step {duration}-minute instructional speech on '{topic}'.",
        "debate": "Write a {duration}-minute debate speech about '{topic}' with strong arguments and counterpoints.",
    }

    audience_guidance = {
        "general": "Make the speech accessible to a general audience with no specialized knowledge.",
        "experts": "Include technical depth suitable for experts in the field.",
        "children": "Use simple, engaging language and examples suitable for kids.",
        "students": "Be educational and engaging for a student audience.",
        "executives": "Focus on strategic implications and leadership perspectives.",
    }

    base_prompt = style_templates.get(emotion, style_templates["formal"]).format(topic=topic, duration=duration)
    audience_note = audience_guidance.get(audience, "")
    
    final_prompt = (
        f"{base_prompt}\n\n"
        f"{audience_note}\n\n"
        f"Structure the speech with an introduction, body, and conclusion.\n"
        f"Use engaging transitions, rhetorical devices, and paragraph breaks.\n"
        f"Aim for roughly {duration * 130} words."
    )
    
    return final_prompt

In [12]:
def generate_speech(topic: str, duration: int, emotion: str, audience: str):
    # Build prompt based on input
    prompt = build_prompt(topic, duration, emotion, audience)
    
    # Groq API call
    api_key = "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"
    client = Groq(api_key=api_key)
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=2048,
        top_p=1,
        stream=True
    )
    
    # Collect and display streaming output
    speech = ""
    for chunk in completion:
        speech += chunk.choices[0].delta.content or ""
    return speech


In [13]:
import re
import os

def sanitize_filename(text):
    # Replace spaces with underscores and remove special characters
    return re.sub(r'[^a-zA-Z0-9_]', '', text.replace(" ", "_"))

topic_input = widgets.Text(value="Recent Trends in AI", description="Topic:")
emotion_dropdown = widgets.Dropdown(
    options=["formal", "casual", "motivational", "persuasive", "instructional", "debate"],
    value="instructional", description="Emotion:"
)
duration_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description="Duration (min):")
audience_dropdown = widgets.Dropdown(
    options=["general", "experts", "children", "students", "executives"],
    value="students", description="Audience:"
)

display(topic_input, emotion_dropdown, duration_slider, audience_dropdown)

# 5. Button to trigger process
generate_button = widgets.Button(description="Generate & Speak")
output = widgets.Output()

def on_button_click(b):
    with output:
        print("Generating speech...\n")
        speech_text = generate_speech(topic_input.value, duration_slider.value, emotion_dropdown.value, audience_dropdown.value)
        print(speech_text)
        speak_text(speech_text)
        print("Speech spoken aloud!")


        folder_path = "speech"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Generate filename from topic
        topic_clean = sanitize_filename(topic_input.value)
        filename = f"speech_{topic_clean}.txt"
        file_path = os.path.join(folder_path, filename)

        # Ensure uniqueness (e.g., if file already exists, add a counter)
        counter = 1
        base_filename = filename
        while os.path.exists(file_path):
            filename = f"{base_filename[:-4]}_{counter}.txt"
            file_path = os.path.join(folder_path, filename)
            counter += 1

        # Save speech to uniquely named .txt file
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(speech_text)
        print(f"Speech saved as '{file_path}'")

generate_button.on_click(on_button_click)
display(generate_button, output)


Text(value='Recent Trends in AI', description='Topic:')

Dropdown(description='Emotion:', index=4, options=('formal', 'casual', 'motivational', 'persuasive', 'instruct…

IntSlider(value=3, description='Duration (min):', max=10, min=1)

Dropdown(description='Audience:', index=3, options=('general', 'experts', 'children', 'students', 'executives'…

Button(description='Generate & Speak', style=ButtonStyle())

Output()